##### Import Necessary Libraries

In [1]:
import pandas as pd
import yfinance as yf
import datetime
from datetime import date, timedelta

In [2]:
today = date.today()

d1 = today.strftime("%Y-%m-%d")
end_date = d1

d2 = today - timedelta(days=730)
d2 = d2.strftime("%Y-%m-%d")
start_date = d2

data = yf.download("BTC-USD",
                   start=start_date,
                   end=end_date,
                   progress=False)

print(data.head())

                    Open          High           Low         Close  \
Date                                                                 
2022-10-05  20335.900391  20343.748047  19801.800781  20160.716797   
2022-10-06  20161.039062  20408.392578  19900.087891  19955.443359   
2022-10-07  19957.558594  20041.085938  19395.792969  19546.849609   
2022-10-08  19546.328125  19601.695312  19299.414062  19416.568359   
2022-10-09  19417.478516  19542.539062  19349.259766  19446.425781   

               Adj Close       Volume  
Date                                   
2022-10-05  20160.716797  33223790572  
2022-10-06  19955.443359  34711412966  
2022-10-07  19546.849609  29227315390  
2022-10-08  19416.568359  16437423167  
2022-10-09  19446.425781  16837262532  


In [4]:
data["Date"] = data.index
data = data[["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"]]
data.reset_index(drop=True, inplace=True)
print(data.head())

        Date          Open          High           Low         Close  \
0 2022-10-05  20335.900391  20343.748047  19801.800781  20160.716797   
1 2022-10-06  20161.039062  20408.392578  19900.087891  19955.443359   
2 2022-10-07  19957.558594  20041.085938  19395.792969  19546.849609   
3 2022-10-08  19546.328125  19601.695312  19299.414062  19416.568359   
4 2022-10-09  19417.478516  19542.539062  19349.259766  19446.425781   

      Adj Close       Volume  
0  20160.716797  33223790572  
1  19955.443359  34711412966  
2  19546.849609  29227315390  
3  19416.568359  16437423167  
4  19446.425781  16837262532  


In [5]:
data.shape

(730, 7)

In [8]:
# Now let's visualize the change in bitcoint prices till today by using a candlestick chart
import plotly.graph_objects as go 
fig = go.Figure(data=[go.Candlestick(x=data["Date"],
                                     open=data["Open"],
                                     high=data["High"],
                                     low=data["Low"],
                                     close=data["Close"])])
fig.update_layout(title="Bitcoin Price Analysis",
                  xaxis_rangeslider_visible=True)
fig.show()

In [11]:
#The Close column in the dataset contains the values we need to predict. So, let’s have a look at the correlation
#  of all the columns in the data concerning the Close column:
correlation = data.corr()
print(correlation["Close"].sort_values(ascending=False))

Close        1.000000
Adj Close    1.000000
High         0.999247
Low          0.998913
Open         0.997853
Date         0.921770
Volume       0.345872
Name: Close, dtype: float64


##### Cryptocurrency Price Prediction Model

In [16]:
# The AutoTS library in Python is one of the best libraries for time series analysis.
#  So here I will be using the AutoTS library to predict the bitcoin prices for the next 30 days:
from autots import AutoTS
model = AutoTS(forecast_length=30, frequency='infer', ensemble='simple')
model = model.fit(data, date_col='Date', value_col='Close', id_col=None)
prediction = model.predict()
forecast = prediction.forecast
print(forecast)

Using 2 cpus for n_jobs.
Data frequency is: D, used frequency is: D
Model Number: 1 with model ARIMA in generation 0 of 20
Model Number: 2 with model AverageValueNaive in generation 0 of 20
Model Number: 3 with model AverageValueNaive in generation 0 of 20
Model Number: 4 with model AverageValueNaive in generation 0 of 20
Model Number: 5 with model DatepartRegression in generation 0 of 20
Model Number: 6 with model DatepartRegression in generation 0 of 20
Model Number: 7 with model DatepartRegression in generation 0 of 20


c:\Users\thars\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning:

The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.

c:\Users\thars\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.

c:\Users\thars\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html



Model Number: 8 with model DatepartRegression in generation 0 of 20



c:\Users\thars\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 1/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - loss: 0.4238
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.4090
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.4022
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3877 
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3673
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3507
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3387
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3371
Epoch 9/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3390
Epoch 10/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3421
Epoch 11/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3354
Epoch 12/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3362
Epoch 13/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3441
Epoch 14/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3358
Epoch 15/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3381
Epoch 16/50
22/22

...

c:\Users\thars\anaconda3\Lib\site-packages\autots\tools\transform.py:5608: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

c:\Users\thars\anaconda3\Lib\site-packages\autots\tools\transform.py:5608: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

c:\Users\thars\anaconda3\Lib\site-packages\autots\tools\transform.py:5608: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent

8 - Ensemble with avg smape 4.8: 


c:\Users\thars\anaconda3\Lib\site-packages\autots\evaluator\auto_model.py:2462: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

c:\Users\thars\anaconda3\Lib\site-packages\autots\evaluator\auto_model.py:2462: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



                   Close
2024-10-04  60784.896994
2024-10-05  60979.934747
2024-10-06  61178.065804
2024-10-07  61832.168704
2024-10-08  62331.639138
2024-10-09  62336.932982
2024-10-10  62435.613005
2024-10-11  62610.947398
2024-10-12  62244.991155
2024-10-13  61857.177142
2024-10-14  61928.181007
2024-10-15  62292.995535
2024-10-16  62538.220003
2024-10-17  63209.057316
2024-10-18  63225.895116
2024-10-19  63431.708278
2024-10-20  64020.470362
2024-10-21  63942.314839
2024-10-22  65231.065398
2024-10-23  65529.884968
2024-10-24  65772.835776
2024-10-25  65519.960698
2024-10-26  65304.482005
2024-10-27  65693.437597
2024-10-28  65696.507666
2024-10-29  65711.555038
2024-10-30  65588.434142
2024-10-31  65210.163893
2024-11-01  65138.454386
2024-11-02  64984.103939


c:\Users\thars\anaconda3\Lib\site-packages\autots\tools\transform.py:5608: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

c:\Users\thars\anaconda3\Lib\site-packages\autots\tools\transform.py:5608: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

